In [1]:
!pip install transformers datasets huggingface_hub torch accelerate -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's d

**wait!!!**

In [69]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoModelForMaskedLM,
)
from typing import List, Tuple, Dict, Any, Optional
import re
import warnings
from copy import deepcopy

warnings.filterwarnings("ignore")

# ============================================================================
# CONFIGURATION
# ============================================================================

BASE_MODEL = "IRIIS-RESEARCH/RoBERTa_Nepali_125M"

# Model Hub IDs
GED_MODEL_HUB_ID = "DipeshChaudhary/roberta-nepali-sequence-ged"
BINARY_MODEL_HUB_ID = "DipeshChaudhary/nepali-gec-binary-detector"
ERROR_TYPE_MODEL_HUB_ID = "DipeshChaudhary/nepali-gec-error-type-classifier"
MLM_MODEL_HUB_ID = "DipeshChaudhary/nepali-mlm-guesser-finetuned-model-1"

# CORRECT Label Mapping for Error Type Model
ERROR_ID2LABEL = {
    0: '$DELETE',
    1: '$REPLACE',
    2: '$APPEND',
    3: '$SWAP_NEXT',
    4: '$SWAP_PREV',
    5: '$MERGE_NEXT',
    6: '$MERGE_PREV'
}

ERROR_LABEL2ID = {v: k for k, v in ERROR_ID2LABEL.items()}

# Optimal Thresholds
BINARY_THRESHOLD = 0.42

ERROR_TYPE_THRESHOLDS = {
    '$DELETE': 0.05,
    '$REPLACE': 0.51,
    '$APPEND': 0.45,
    '$SWAP_NEXT': 0.50,
    '$SWAP_PREV': 0.51,
    '$MERGE_NEXT': 0.05,
    '$MERGE_PREV': 0.05,
}

# Tags categorized by reliability
RELIABLE_TAGS = {'$REPLACE', '$APPEND', '$SWAP_NEXT', '$SWAP_PREV'}
UNRELIABLE_TAGS = {'$DELETE', '$MERGE_NEXT', '$MERGE_PREV'}

# Pipeline parameters
MAX_CORRECTION_ITERATIONS = 10
MLM_TOP_K_SUGGESTIONS = 5  # Increased from 3 for aggressive fallback

# ============================================================================
# MODEL LOADING
# ============================================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Using device: {device}")

# Tokenizers
ged_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
gector_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_prefix_space=True)

# Models
print("📦 Loading models...")
ged_model = AutoModelForSequenceClassification.from_pretrained(GED_MODEL_HUB_ID).to(device)
binary_model = AutoModelForTokenClassification.from_pretrained(BINARY_MODEL_HUB_ID).to(device)
error_model = AutoModelForTokenClassification.from_pretrained(ERROR_TYPE_MODEL_HUB_ID).to(device)
mlm_model = AutoModelForMaskedLM.from_pretrained(MLM_MODEL_HUB_ID).to(device)

ged_model.eval()
binary_model.eval()
error_model.eval()
mlm_model.eval()
print("✅ All models loaded successfully!")

# ============================================================================
# PREPROCESSING & POSTPROCESSING
# ============================================================================

def preprocess_sentence(sentence: str) -> str:
    """Separates punctuation for better GEC token alignment."""
    sentence = re.sub(r'([।?,!])', r' \1', sentence)
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

def postprocess_sentence(tokens: List[str]) -> str:
    """Re-merges separated punctuation."""
    sentence = " ".join(tokens)
    sentence = re.sub(r'\s+([।?,!])', r'\1', sentence).strip()
    return sentence

# ============================================================================
# GED & BINARY VALIDATION
# ============================================================================

def check_sentence_correctness(sentence: str) -> Tuple[bool, float]:
    """Check if sentence is correct using GED model."""
    inputs = ged_tokenizer(sentence, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        logits = ged_model(**inputs).logits
    probs = torch.softmax(logits, dim=-1)[0]
    is_correct = torch.argmax(probs).item() == 0
    confidence = probs[0 if is_correct else 1].item()
    return is_correct, confidence

def validate_with_binary_model(sentence: str) -> Tuple[bool, List[int]]:
    """Validate sentence using token-level binary classifier."""
    inputs = gector_tokenizer(sentence, return_tensors="pt", truncation=True).to(device)
    
    with torch.no_grad():
        binary_logits = binary_model(**inputs).logits
    
    binary_probs = torch.softmax(binary_logits, dim=-1)[0]
    
    # Map tokens to words
    word_ids = inputs.word_ids(batch_index=0)
    word_start_token_map = {}
    for token_idx, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx not in word_start_token_map:
            word_start_token_map[word_idx] = token_idx
    
    error_positions = []
    for word_idx in sorted(word_start_token_map.keys()):
        token_idx = word_start_token_map[word_idx]
        binary_error_prob = binary_probs[token_idx, 1].item()
        
        if binary_error_prob > BINARY_THRESHOLD:
            error_positions.append(word_idx)
    
    all_correct = len(error_positions) == 0
    return all_correct, error_positions

def is_sentence_fully_correct(sentence: str) -> Tuple[bool, str]:
    """Validate sentence using BOTH GED and Binary models."""
    ged_correct, ged_conf = check_sentence_correctness(sentence)
    binary_correct, error_positions = validate_with_binary_model(sentence)
    
    if ged_correct and binary_correct:
        return True, f"✓ Both models agree (GED: {ged_conf:.3f})"
    elif not ged_correct and not binary_correct:
        return False, f"✗ Both models detect errors (GED: {ged_conf:.3f}, Binary: {len(error_positions)} errors)"
    elif not ged_correct:
        return False, f"✗ GED detects error (conf: {ged_conf:.3f})"
    else:
        return False, f"✗ Binary detects {len(error_positions)} error(s) at positions: {error_positions}"

# ============================================================================
# MLM GUESSING (Multi-token strategy)
# ============================================================================

def get_mlm_suggestions(sentence: str, mask_positions: List[int], k: int = MLM_TOP_K_SUGGESTIONS, verbose: bool = False) -> List[List[str]]:
    """Get MLM suggestions with multi-mask strategy for compound words."""
    if not mask_positions:
        return []
    
    token_list = sentence.split()
    valid_positions = [pos for pos in mask_positions if 0 <= pos <= len(token_list)]
    if not valid_positions:
        return []
    
    all_suggestions_per_position = []
    
    for pos in valid_positions:
        position_suggestions = []
        
        if verbose:
            print(f"      🔍 MLM for position {pos} (word: '{token_list[pos] if pos < len(token_list) else 'APPEND'}')")
        
        # Strategy 1: Single mask
        single_mask_suggestions = _get_single_mask_suggestions(token_list, pos, k)
        if verbose and single_mask_suggestions:
            print(f"        Single mask: {single_mask_suggestions[:3]}")
        position_suggestions.extend(single_mask_suggestions)
        
        # Strategy 2: Double mask
        double_mask_suggestions = _get_multi_mask_suggestions(token_list, pos, num_masks=2, k=k)
        if verbose and double_mask_suggestions:
            print(f"        Double mask: {double_mask_suggestions[:3]}")
        position_suggestions.extend(double_mask_suggestions)
        
        # Strategy 3: Triple mask
        triple_mask_suggestions = _get_multi_mask_suggestions(token_list, pos, num_masks=3, k=k)
        if verbose and triple_mask_suggestions:
            print(f"        Triple mask: {triple_mask_suggestions[:3]}")
        position_suggestions.extend(triple_mask_suggestions)
        
        # Remove duplicates and keep top-k
        seen = set()
        unique_suggestions = []
        for sugg in position_suggestions:
            if sugg and sugg not in seen:
                seen.add(sugg)
                unique_suggestions.append(sugg)
                if len(unique_suggestions) >= k:
                    break
        
        if verbose:
            print(f"        ✓ Final suggestions: {unique_suggestions}")
        
        all_suggestions_per_position.append(unique_suggestions[:k])
    
    return all_suggestions_per_position

def _get_single_mask_suggestions(token_list: List[str], position: int, k: int) -> List[str]:
    """Get suggestions with a single [MASK] token."""
    temp_list = list(token_list)
    
    if position < len(temp_list):
        temp_list[position] = gector_tokenizer.mask_token
    elif position == len(temp_list):
        temp_list.append(gector_tokenizer.mask_token)
    else:
        return []
    
    masked_sentence = " ".join(temp_list)
    return _predict_masked_tokens(masked_sentence, k)

def _get_multi_mask_suggestions(token_list: List[str], position: int, num_masks: int, k: int) -> List[str]:
    """Get suggestions by placing multiple consecutive [MASK] tokens."""
    temp_list = list(token_list)
    
    if position < len(temp_list):
        temp_list[position:position+1] = [gector_tokenizer.mask_token] * num_masks
    elif position == len(temp_list):
        temp_list.extend([gector_tokenizer.mask_token] * num_masks)
    else:
        return []
    
    masked_sentence = " ".join(temp_list)
    return _predict_masked_tokens(masked_sentence, k, expect_multi_token=True)

def _predict_masked_tokens(masked_sentence: str, k: int, expect_multi_token: bool = False) -> List[str]:
    """Core MLM prediction function."""
    inputs = gector_tokenizer(masked_sentence, return_tensors='pt', truncation=True).to(device)
    mask_token_indices = torch.where(inputs["input_ids"] == gector_tokenizer.mask_token_id)[1]
    
    if mask_token_indices.numel() == 0:
        return []
    
    with torch.no_grad():
        logits = mlm_model(**inputs).logits
    
    suggestions = []
    
    if expect_multi_token and mask_token_indices.numel() > 1:
        num_masks = mask_token_indices.numel()
        first_mask_logits = logits[0, mask_token_indices[0], :]
        top_first = torch.topk(first_mask_logits, min(k * 2, 50)).indices.tolist()
        
        for first_token_id in top_first:
            first_decoded = gector_tokenizer.decode(first_token_id).strip()
            
            if not _is_valid_token(first_decoded):
                continue
            
            remaining_parts = []
            for mask_idx in mask_token_indices[1:]:
                mask_logits = logits[0, mask_idx, :]
                top_token_id = torch.argmax(mask_logits).item()
                decoded = gector_tokenizer.decode(top_token_id).strip()
                
                if decoded.startswith('##'):
                    decoded = decoded[2:]
                
                if decoded and len(decoded) > 0:
                    remaining_parts.append(decoded)
            
            if remaining_parts:
                full_word = first_decoded + "".join(remaining_parts)
            else:
                full_word = first_decoded
            
            if full_word and len(full_word) > 0:
                suggestions.append(full_word)
            
            if len(suggestions) >= k:
                break
    else:
        for mask_idx in mask_token_indices:
            mask_logits = logits[0, mask_idx, :]
            top_k_tokens = torch.topk(mask_logits, k * 3).indices.tolist()
            
            for token_id in top_k_tokens:
                decoded = gector_tokenizer.decode(token_id).strip()
                
                if _is_valid_token(decoded):
                    suggestions.append(decoded)
                
                if len(suggestions) >= k:
                    break
            
            if suggestions:
                break
    
    return suggestions[:k]

def _is_valid_token(token: str) -> bool:
    """Check if a token is valid for use as a correction."""
    if not token or len(token) < 1:
        return False
    
    invalid_tokens = {'(', ')', '[', ']', '{', '}', '<', '>', '"', "'", '`', ',', '.'}
    if token in invalid_tokens:
        return False
    
    if token.startswith('##'):
        return False
    
    return True

def apply_mlm_to_positions(tokens: List[str], positions: List[int], suggestions: List[List[str]]) -> str:
    """Apply MLM suggestions to specific positions."""
    result = list(tokens)
    
    for pos, sugg_list in zip(positions, suggestions):
        if sugg_list and 0 <= pos < len(result):
            suggestion = sugg_list[0]
            if _is_valid_token(suggestion):
                result[pos] = suggestion
    
    return " ".join(result)

# ============================================================================
# TOKEN CLASSIFICATION & ERROR DETECTION
# ============================================================================

def get_token_predictions(sentence: str, verbose: bool = False) -> List[Dict[str, Any]]:
    """Get comprehensive token-level predictions."""
    inputs = gector_tokenizer(sentence, return_tensors="pt", truncation=True).to(device)
    
    with torch.no_grad():
        binary_logits = binary_model(**inputs).logits
        error_logits = error_model(**inputs).logits
    
    binary_probs = torch.softmax(binary_logits, dim=-1)[0]
    error_probs = torch.softmax(error_logits, dim=-1)[0]
    
    word_ids = inputs.word_ids(batch_index=0)
    word_start_token_map = {}
    for token_idx, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx not in word_start_token_map:
            word_start_token_map[word_idx] = token_idx
    
    tokens = sentence.split()
    predictions = []
    
    for word_idx in sorted(word_start_token_map.keys()):
        if word_idx >= len(tokens):
            continue
        
        token_idx = word_start_token_map[word_idx]
        token = tokens[word_idx]
        
        binary_error_prob = binary_probs[token_idx, 1].item()
        
        error_type_probs = {}
        for error_id, error_tag in ERROR_ID2LABEL.items():
            error_type_probs[error_tag] = error_probs[token_idx, error_id].item()
        
        candidate_tags = []
        for tag, prob in error_type_probs.items():
            threshold = ERROR_TYPE_THRESHOLDS[tag]
            if prob > threshold:
                candidate_tags.append((tag, prob))
        
        candidate_tags.sort(key=lambda x: x[1], reverse=True)
        
        is_error = binary_error_prob > BINARY_THRESHOLD
        primary_tag = candidate_tags[0][0] if candidate_tags else None
        
        pred = {
            'word_idx': word_idx,
            'token': token,
            'binary_error_prob': binary_error_prob,
            'error_type_probs': error_type_probs,
            'candidate_tags': candidate_tags,
            'primary_tag': primary_tag,
            'is_error': is_error,
            'is_reliable': primary_tag in RELIABLE_TAGS if primary_tag else False
        }
        
        predictions.append(pred)
        
        if verbose and is_error:
            print(f"  ⚠️  Token '{token}' @ pos {word_idx}")
            print(f"      Binary: {binary_error_prob:.3f} > {BINARY_THRESHOLD}")
            if primary_tag:
                print(f"      Primary: {primary_tag} ({error_type_probs[primary_tag]:.3f})")
    
    return predictions

# ============================================================================
# CORRECTION STRATEGIES
# ============================================================================

def apply_tag_based_correction(tokens: List[str], corrections: List[Dict[str, Any]], 
                                use_mlm_for_unreliable: bool = False) -> str:
    """Apply corrections based on predicted tags with proper phase handling."""
    result_tokens = list(tokens)
    valid_corrections = [c for c in corrections if c['primary_tag'] is not None]
    
    if not valid_corrections:
        return " ".join(result_tokens)
    
    swaps = []
    deletes_merges = []
    replaces_appends = []
    
    for corr in valid_corrections:
        tag = corr['primary_tag']
        if tag in ['$SWAP_NEXT', '$SWAP_PREV']:
            swaps.append(corr)
        elif tag in ['$DELETE', '$MERGE_NEXT', '$MERGE_PREV']:
            deletes_merges.append(corr)
        else:
            replaces_appends.append(corr)
    
    # PHASE 1: Swaps
    for corr in swaps:
        idx = corr['word_idx']
        tag = corr['primary_tag']
        
        if tag == '$SWAP_NEXT' and idx + 1 < len(result_tokens):
            result_tokens[idx], result_tokens[idx + 1] = result_tokens[idx + 1], result_tokens[idx]
        elif tag == '$SWAP_PREV' and idx > 0:
            result_tokens[idx], result_tokens[idx - 1] = result_tokens[idx - 1], result_tokens[idx]
    
    # PHASE 2: Deletions and Merges
    for corr in sorted(deletes_merges, key=lambda x: x['word_idx'], reverse=True):
        idx = corr['word_idx']
        tag = corr['primary_tag']
        
        if idx >= len(result_tokens):
            continue
        
        if use_mlm_for_unreliable:
            continue
        
        if tag == '$DELETE':
            result_tokens.pop(idx)
        elif tag == '$MERGE_NEXT' and idx + 1 < len(result_tokens):
            result_tokens[idx] = result_tokens[idx] + result_tokens.pop(idx + 1)
        elif tag == '$MERGE_PREV' and idx > 0:
            result_tokens[idx - 1] = result_tokens[idx - 1] + result_tokens.pop(idx)
    
    # PHASE 3: Replacements and Appends
    offset = 0
    for corr in sorted(replaces_appends, key=lambda x: x['word_idx']):
        original_idx = corr['word_idx']
        tag = corr['primary_tag']
        current_idx = original_idx + offset
        
        if current_idx >= len(result_tokens):
            continue
        
        if tag == '$REPLACE':
            suggestions = get_mlm_suggestions(" ".join(result_tokens), [current_idx], k=5, verbose=False)
            if suggestions and suggestions[0]:
                for sugg in suggestions[0]:
                    if _is_valid_token(sugg):
                        result_tokens[current_idx] = sugg
                        break
        
        elif tag == '$APPEND':
            suggestions = get_mlm_suggestions(" ".join(result_tokens), [current_idx + 1], k=5)
            if suggestions and suggestions[0]:
                for sugg in suggestions[0]:
                    if _is_valid_token(sugg):
                        result_tokens.insert(current_idx + 1, sugg)
                        offset += 1
                        break
    
    return " ".join(result_tokens)

def generate_correction_candidates(tokens: List[str], predictions: List[Dict[str, Any]]) -> List[Tuple[str, str]]:
    """Generate multiple correction candidates with strategies."""
    candidates = []
    
    error_predictions = [p for p in predictions if p['is_error']]
    if not error_predictions:
        return [((" ".join(tokens), "no_errors"))]
    
    valid_predictions = [p for p in error_predictions if p['primary_tag'] is not None]
    
    # Strategy 1: Primary tags
    if valid_predictions:
        corrected = apply_tag_based_correction(tokens, valid_predictions, use_mlm_for_unreliable=False)
        candidates.append((corrected, "primary_tags"))
    
    # Strategy 2: Forced swap
    swap_corrections = [p for p in valid_predictions if 'SWAP' in (p['primary_tag'] or '')]
    non_swap_errors = [p for p in valid_predictions if 'SWAP' not in (p['primary_tag'] or '')]
    
    if len(swap_corrections) == 1 and non_swap_errors:
        swap_idx = swap_corrections[0]['word_idx']
        for other in non_swap_errors:
            other_idx = other['word_idx']
            if abs(swap_idx - other_idx) == 1:
                temp_tokens = list(tokens)
                if swap_idx < other_idx:
                    temp_tokens[swap_idx], temp_tokens[other_idx] = temp_tokens[other_idx], temp_tokens[swap_idx]
                else:
                    temp_tokens[other_idx], temp_tokens[swap_idx] = temp_tokens[swap_idx], temp_tokens[other_idx]
                
                remaining = [p for p in valid_predictions if p['word_idx'] not in [swap_idx, other_idx]]
                corrected = apply_tag_based_correction(temp_tokens, remaining, use_mlm_for_unreliable=False)
                candidates.append((corrected, "forced_swap"))
                break
    
    # Strategy 3: MLM for unreliable
    unreliable_predictions = [p for p in valid_predictions if not p['is_reliable']]
    if unreliable_predictions:
        unreliable_positions = [p['word_idx'] for p in unreliable_predictions]
        suggestions = get_mlm_suggestions(" ".join(tokens), unreliable_positions)
        
        if suggestions and len(suggestions) == len(unreliable_positions):
            corrected = apply_mlm_to_positions(tokens, unreliable_positions, suggestions)
            reliable_corrections = [p for p in valid_predictions if p['is_reliable']]
            if reliable_corrections:
                corrected = apply_tag_based_correction(corrected.split(), reliable_corrections, use_mlm_for_unreliable=False)
            candidates.append((corrected, "mlm_unreliable"))
    
    # Strategy 4: Full MLM
    all_error_positions = [p['word_idx'] for p in error_predictions]
    if len(all_error_positions) > 1:
        suggestions = get_mlm_suggestions(" ".join(tokens), all_error_positions)
        if suggestions and len(suggestions) == len(all_error_positions):
            corrected = apply_mlm_to_positions(tokens, all_error_positions, suggestions)
            candidates.append((corrected, "full_mlm"))
    
    # Remove duplicates
    seen = set()
    unique_candidates = []
    for sent, strategy in candidates:
        if sent not in seen:
            seen.add(sent)
            unique_candidates.append((sent, strategy))
    
    return unique_candidates

# ============================================================================
# MAIN PIPELINE (WITH AGGRESSIVE MLM FALLBACK)
# ============================================================================

def correct_sentence(sentence: str, verbose: bool = True) -> str:
    """Complete GEC pipeline with aggressive MLM fallback for GED-only errors."""
    preprocessed = preprocess_sentence(sentence)
    
    if verbose:
        print("\n" + "="*80)
        print(f"🔍 INPUT: {sentence}")
        print(f"🔧 PREPROCESSED: {preprocessed}")
        print("="*80)
    
    current_sentence = preprocessed
    iteration = 0
    correction_history = []
    seen_sentences = {current_sentence}
    previous_error_count = float('inf')
    
    while iteration < MAX_CORRECTION_ITERATIONS:
        iteration += 1
        
        if verbose:
            print(f"\n{'─'*80}")
            print(f"🔄 ITERATION {iteration}/{MAX_CORRECTION_ITERATIONS}")
            print(f"{'─'*80}")
        
        is_correct, validation_msg = is_sentence_fully_correct(current_sentence)
        
        if verbose:
            print(f"🔍 Validation: {validation_msg}")
        
        if is_correct:
            if verbose:
                print(f"✅ CORRECTION COMPLETE after {iteration} iteration(s)!")
            break
        
        predictions = get_token_predictions(current_sentence, verbose=verbose)
        error_predictions = [p for p in predictions if p['is_error']]
        
        # CRITICAL FIX: GED-only error scenario - EXHAUSTIVE SEARCH
        if not error_predictions:
            if verbose:
                print("⚠️  Binary found no errors, but GED detected issues.")
                print("🔄 Attempting AGGRESSIVE MLM on ALL token positions...")
            
            tokens = current_sentence.split()
            all_valid_candidates = []  # Store (sentence, confidence, strategy, position)
            
            # PHASE 1: Single-position MLM (1 mask, 2 masks, 3 masks)
            if verbose:
                print(f"\n   📍 PHASE 1: Testing ALL {len(tokens)} positions with single/double/triple masks")
            
            for pos in range(len(tokens)):
                if verbose:
                    print(f"      Position {pos}/{len(tokens)-1} ('{tokens[pos]}')")
                
                # Try single mask
                suggestions_single = get_mlm_suggestions(current_sentence, [pos], k=5, verbose=False)
                
                if suggestions_single and suggestions_single[0]:
                    for sugg in suggestions_single[0]:
                        test_tokens = list(tokens)
                        test_tokens[pos] = sugg
                        test_sentence = " ".join(test_tokens)
                        
                        if test_sentence in seen_sentences:
                            continue
                        
                        is_valid, reason = is_sentence_fully_correct(test_sentence)
                        
                        if is_valid:
                            # Get GED confidence
                            ged_correct, ged_conf = check_sentence_correctness(test_sentence)
                            all_valid_candidates.append({
                                'sentence': test_sentence,
                                'confidence': ged_conf,
                                'strategy': f"1mask_pos{pos}",
                                'position': pos,
                                'original': tokens[pos],
                                'replacement': sugg
                            })
                            if verbose:
                                print(f"         ✅ 1-mask: '{tokens[pos]}' → '{sugg}' (conf: {ged_conf:.3f})")
            
            # PHASE 2: Two-position MLM (adjacent pairs)
            if verbose:
                print(f"\n   📍 PHASE 2: Testing adjacent position pairs ({len(tokens)-1} pairs)")
            
            for pos in range(len(tokens) - 1):
                if verbose:
                    print(f"      Pair [{pos},{pos+1}] ('{tokens[pos]}', '{tokens[pos+1]}')")
                
                suggestions_pair = get_mlm_suggestions(current_sentence, [pos, pos + 1], k=3, verbose=False)
                
                if suggestions_pair and len(suggestions_pair) == 2:
                    # Try all combinations of suggestions
                    for sugg1 in suggestions_pair[0]:
                        for sugg2 in suggestions_pair[1]:
                            test_tokens = list(tokens)
                            test_tokens[pos] = sugg1
                            test_tokens[pos + 1] = sugg2
                            test_sentence = " ".join(test_tokens)
                            
                            if test_sentence in seen_sentences:
                                continue
                            
                            is_valid, reason = is_sentence_fully_correct(test_sentence)
                            
                            if is_valid:
                                ged_correct, ged_conf = check_sentence_correctness(test_sentence)
                                all_valid_candidates.append({
                                    'sentence': test_sentence,
                                    'confidence': ged_conf,
                                    'strategy': f"2mask_pos{pos}-{pos+1}",
                                    'position': pos,
                                    'original': f"{tokens[pos]} {tokens[pos+1]}",
                                    'replacement': f"{sugg1} {sugg2}"
                                })
                                if verbose:
                                    print(f"         ✅ 2-mask: '{tokens[pos]} {tokens[pos+1]}' → '{sugg1} {sugg2}' (conf: {ged_conf:.3f})")
            
            # PHASE 3: Three-position MLM (adjacent triplets)
            if len(tokens) > 2:
                if verbose:
                    print(f"\n   📍 PHASE 3: Testing adjacent position triplets ({len(tokens)-2} triplets)")
                
                for pos in range(len(tokens) - 2):
                    if verbose:
                        print(f"      Triplet [{pos},{pos+1},{pos+2}] ('{tokens[pos]}', '{tokens[pos+1]}', '{tokens[pos+2]}')")
                    
                    suggestions_triple = get_mlm_suggestions(current_sentence, [pos, pos + 1, pos + 2], k=2, verbose=False)
                    
                    if suggestions_triple and len(suggestions_triple) == 3:
                        # Try top combinations
                        for sugg1 in suggestions_triple[0][:2]:
                            for sugg2 in suggestions_triple[1][:2]:
                                for sugg3 in suggestions_triple[2][:2]:
                                    test_tokens = list(tokens)
                                    test_tokens[pos] = sugg1
                                    test_tokens[pos + 1] = sugg2
                                    test_tokens[pos + 2] = sugg3
                                    test_sentence = " ".join(test_tokens)
                                    
                                    if test_sentence in seen_sentences:
                                        continue
                                    
                                    is_valid, reason = is_sentence_fully_correct(test_sentence)
                                    
                                    if is_valid:
                                        ged_correct, ged_conf = check_sentence_correctness(test_sentence)
                                        all_valid_candidates.append({
                                            'sentence': test_sentence,
                                            'confidence': ged_conf,
                                            'strategy': f"3mask_pos{pos}-{pos+1}-{pos+2}",
                                            'position': pos,
                                            'original': f"{tokens[pos]} {tokens[pos+1]} {tokens[pos+2]}",
                                            'replacement': f"{sugg1} {sugg2} {sugg3}"
                                        })
                                        if verbose:
                                            print(f"         ✅ 3-mask: (conf: {ged_conf:.3f})")
            
            # SELECT BEST CANDIDATE BY HIGHEST CONFIDENCE
            if all_valid_candidates:
                # Sort by confidence (descending)
                all_valid_candidates.sort(key=lambda x: x['confidence'], reverse=True)
                
                best = all_valid_candidates[0]
                
                if verbose:
                    print(f"\n   🏆 BEST CANDIDATE SELECTED:")
                    print(f"      Strategy: {best['strategy']}")
                    print(f"      Change: '{best['original']}' → '{best['replacement']}'")
                    print(f"      Confidence: {best['confidence']:.4f}")
                    print(f"      Total valid candidates: {len(all_valid_candidates)}")
                    
                    if len(all_valid_candidates) > 1:
                        print(f"\n   📊 Other candidates:")
                        for i, cand in enumerate(all_valid_candidates[1:6], 1):  # Show top 5
                            print(f"      {i}. [{cand['strategy']}] conf={cand['confidence']:.4f}")
                
                current_sentence = best['sentence']
                seen_sentences.add(current_sentence)
                correction_history.append((iteration, best['strategy'], current_sentence))
                
                if verbose:
                    print(f"\n   ✅ Applied best correction with confidence {best['confidence']:.4f}")
            else:
                if verbose:
                    print(f"\n   ❌ No valid corrections found after exhaustive search")
                    print(f"      Tested {len(tokens)} single positions")
                    print(f"      Tested {len(tokens)-1} position pairs")
                    print(f"      Tested {max(0, len(tokens)-2)} position triplets")
                    print("⚠️  Stopping.")
                break
            
            continue  # Skip to next iteration
        
        if verbose:
            print(f"\n📊 Found {len(error_predictions)} error token(s)")
        
        if len(error_predictions) > previous_error_count * 1.5:
            if verbose:
                print(f"⚠️  Error count increased significantly. Stopping.")
            break
        
        previous_error_count = len(error_predictions)
        
        tokens = current_sentence.split()
        candidates = generate_correction_candidates(tokens, predictions)
        
        if verbose:
            print(f"\n💡 Generated {len(candidates)} correction candidate(s)")
        
        best_candidate = None
        best_strategy = None
        
        for candidate, strategy in candidates:
            if candidate in seen_sentences:
                if verbose:
                    print(f"   🔄 [{strategy}]: Already seen (skipping)")
                continue
            
            is_valid, reason = is_sentence_fully_correct(candidate)
            
            if verbose:
                status = "✅" if is_valid else "❌"
                print(f"   {status} [{strategy}]: {candidate[:80]}{'...' if len(candidate) > 80 else ''}")
            
            if is_valid:
                best_candidate = candidate
                best_strategy = strategy
                break
        
        if best_candidate:
            current_sentence = best_candidate
            seen_sentences.add(current_sentence)
            correction_history.append((iteration, best_strategy, current_sentence))
            if verbose:
                print(f"\n✓ Applied [{best_strategy}] correction")
        else:
            if iteration >= MAX_CORRECTION_ITERATIONS - 1:
                if verbose:
                    print(f"\n🆘 LAST RESORT: Attempting full MLM masking...")
                
                error_positions = [p['word_idx'] for p in error_predictions]
                suggestions = get_mlm_suggestions(current_sentence, error_positions)
                
                if suggestions and len(suggestions) == len(error_positions):
                    last_attempt = apply_mlm_to_positions(tokens, error_positions, suggestions)
                    is_valid, reason = is_sentence_fully_correct(last_attempt)
                    
                    if verbose:
                        status = "✅" if is_valid else "❌"
                        print(f"   {status} Full MLM result: {last_attempt}")
                    
                    if is_valid:
                        current_sentence = last_attempt
                        break
                
                if verbose:
                    print(f"\n❌ UNABLE TO CORRECT")
                    print(f"   This error pattern is too complex for the current models.")
                
                if len(error_predictions) > 5:
                    if verbose:
                        print(f"   Reverting to original sentence.")
                    current_sentence = preprocessed
            else:
                if candidates:
                    candidate_errors = len([p for p in get_token_predictions(candidates[0][0]) if p['is_error']])
                    if candidate_errors <= len(error_predictions):
                        current_sentence = candidates[0][0]
                        seen_sentences.add(current_sentence)
                        if verbose:
                            print(f"\n⚠️  Using best candidate despite validation failure")
                    else:
                        if verbose:
                            print(f"\n⚠️  Candidate would make things worse. Stopping.")
                        break
    
    final_output = postprocess_sentence(current_sentence.split())
    
    if verbose:
        print("\n" + "="*80)
        print(f"🎯 FINAL OUTPUT: {final_output}")
        if correction_history:
            print(f"📈 Correction path: {' → '.join([f'{s}(iter {i})' for i, s, _ in correction_history])}")
        print("="*80 + "\n")
    
    return final_output

# ============================================================================
# BATCH PROCESSING
# ============================================================================

def correct_sentences_batch(sentences: List[str], verbose: bool = False) -> List[Tuple[str, str]]:
    """Correct multiple sentences."""
    results = []
    
    for i, sent in enumerate(sentences, 1):
        if verbose:
            print(f"\n{'='*80}")
            print(f"Processing sentence {i}/{len(sentences)}")
        
        corrected = correct_sentence(sent, verbose=verbose)
        results.append((sent, corrected))
    
    return results

# ============================================================================
# EXAMPLE USAGE
# ============================================================================

if __name__ == "__main__":
    test_sentences = [
        "नाम मेरो दिपेश हो ।",
        "म स्कुल जान्छु ।",
        "यो किताब मेरो हो ।",
        "डोजर पनि माटोले छोपं गएको छ ।",
        "कार्यालयमा जडान गरिएका आधा दर्जनभन्दा बढी क्यामेराले चौबिसै घन्टा काम गर्न सक्ी सौर्य ऊर्जामा जडान गरिएको हो।",
        "तालिमको लागि बोलाउँदा पनि नजाने मुडमा ढुक्क भएर बसिरहेको थिएँ ।",
        "दुर्घटनाबाजा बजाएर आउँदैन मानिसकै ससानोगल्तीले हुने हो ।",
        "भीमदत्त नगरपालिकापछि प्रदेश सरकारको सबैभन्दा धेरै बजेट कृष्णपुर नगरपालिकामा परेको ।",
        "समृद्धिका लागि आर्थिक पातो नै मुख्य हो ।",
        "मेरो नाम दिपेश हो।"
    ]
    
    print("\n" + "🚀"*40)
    print("NEPALI GEC PIPELINE - WITH AGGRESSIVE MLM FALLBACK")
    print("🚀"*40)
    
    for sent in test_sentences:
        corrected = correct_sentence(sent, verbose=True)
    
    print("\n" + "✅"*40)
    print("PROCESSING COMPLETE")
    print("✅"*40)

🔧 Using device: cuda
📦 Loading models...


Some weights of the model checkpoint at DipeshChaudhary/nepali-gec-error-type-classifier were not used when initializing RobertaForTokenClassification: ['loss_fct.ce_loss.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ All models loaded successfully!

🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
NEPALI GEC PIPELINE - WITH AGGRESSIVE MLM FALLBACK
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

🔍 INPUT: नाम मेरो दिपेश हो ।
🔧 PREPROCESSED: नाम मेरो दिपेश हो ।

────────────────────────────────────────────────────────────────────────────────
🔄 ITERATION 1/10
────────────────────────────────────────────────────────────────────────────────
🔍 Validation: ✗ Both models detect errors (GED: 0.893, Binary: 2 errors)
  ⚠️  Token 'नाम' @ pos 0
      Binary: 0.491 > 0.42
      Primary: $APPEND (0.947)
  ⚠️  Token 'मेरो' @ pos 1
      Binary: 0.739 > 0.42
      Primary: $SWAP_NEXT (0.632)

📊 Found 2 error token(s)

💡 Generated 3 correction candidate(s)
   ❌ [primary_tags]: नाम हो, दिपेश मेरो हो ।
   ✅ [forced_swap]: मेरो नाम दिपेश हो ।

✓ Applied [forced_swap] correction

────────────────────────────────────────────────────────────────────────────────
🔄 ITERATION 2/10
────────────────────────────────────────────────────────

In [87]:
if __name__ == "__main__":
    test_sentences = [
        "नाम मेरो दिपेश हो ।",
        "म स्कुल जान्छु ।",
        "यो किताब मेरो हो ।",
        "डोजर पनि माटोले छोपं गएको छ ।",
        "कार्यालयमा जडान गरिएका आधा दर्जनभन्दा बढी क्यामेराले चौबिसै घन्टा काम गर्न सक्ी सौर्य ऊर्जामा जडान गरिएको हो।",
        "तालिमको लागि बोलाउँदा पनि नजाने मुडमा ढुक्क भएर बसिरहेको थिएँ ।",
        "दुर्घटनाबाजा बजाएर आउँदैन मानिसकै ससानोगल्तीले हुने हो ।",
        "भीमदत्त नगरपालिकापछि प्रदेश सरकारको सबैभन्दा धेरै बजेट कृष्णपुर नगरपालिकामा परेको ।",
        "समृद्धिका लागि आर्थिक पातो नै मुख्य हो ।",
        "मेरो नाम दिपेश हो"
    ]
    
    print("\n" + "🚀"*40)
    print("NEPALI GEC PIPELINE - WITH AGGRESSIVE MLM FALLBACK")
    print("🚀"*40)
    pairs = []
    
    for sent in test_sentences:
        corrected = correct_sentence(sent, verbose=True)
        pairs.append((sent, corrected))   # store pair
    
    print("\n" + "✅"*40)
    print("PROCESSING COMPLETE")
    print("✅"*40)
    print("\n" + "✅"*40)
    print("PROCESSING COMPLETE")
    print("✅"*40)


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
NEPALI GEC PIPELINE - WITH AGGRESSIVE MLM FALLBACK
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

🔍 INPUT: नाम मेरो दिपेश हो ।
🔧 PREPROCESSED: नाम मेरो दिपेश हो ।

────────────────────────────────────────────────────────────────────────────────
🔄 ITERATION 1/10
────────────────────────────────────────────────────────────────────────────────
🔍 Validation: ✗ Both models detect errors (GED: 0.893, Binary: 2 errors)
  ⚠️  Token 'नाम' @ pos 0
      Binary: 0.491 > 0.42
      Primary: $APPEND (0.947)
  ⚠️  Token 'मेरो' @ pos 1
      Binary: 0.739 > 0.42
      Primary: $SWAP_NEXT (0.632)

📊 Found 2 error token(s)

💡 Generated 3 correction candidate(s)
   ❌ [primary_tags]: नाम हो, दिपेश मेरो हो ।
   ✅ [forced_swap]: मेरो नाम दिपेश हो ।

✓ Applied [forced_swap] correction

────────────────────────────────────────────────────────────────────────────────
🔄 ITERATION 2/10
────────────────────────────────────────────────────────────────────────────────
🔍 Validat

In [88]:
print("\n" + "📋"*40)
print("Test sentences             ---------------->       CORRECTED SENTENCES")
print("📋"*40)

print(f"| {'Test Sentences':<50} | {'Corrected Sentence'} |")
print("-"*200)

for inc, cor in pairs:
    print("-"*100)
    print(f"{inc:<50} | {cor}")


📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋
Test sentences             ---------------->       CORRECTED SENTENCES
📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋📋
| Test Sentences                                     | Corrected Sentence |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
नाम मेरो दिपेश हो ।                                | मेरो नाम दिपेश हो।
----------------------------------------------------------------------------------------------------
म स्कुल जान्छु ।                                   | म स्कुल जान्छु।
----------------------------------------------------------------------------------------------------
यो किताब मेरो हो ।                                 | यो किताब मेरो हो।
------------------------------------------------------